In [ ]:
# Brain Tumor Recognition

# Vorgehen
1. Data preprocessing
2. Convulutional NN aufbauen
3. CNN trainieren
4. Analysieren
5. Falls möglich: Dense Network anhängen

# Data Preprocessing

In [34]:
import os
import pandas as pd

def label_images(base_dir):
    label_mapping = {"No_Tumor": 0, "Tumor": 1}
    data = []

    # Verwende explizit den backslash \, um den Basisordner zu erstellen
    base_dir = r"C:\Users\leoni\Documents\HSLU\5. Semester\AI Einzelarbeit\Daten"
    print(f"Verwendeter Basisordner: {base_dir}")

    for split in ["Training", "Testing"]:
        split_path = base_dir + '\\' + split  # Explizit den backslash verwenden
        print(f"Überprüfe Ordner: {split_path}")

        # Überprüfen, ob das Verzeichnis existiert
        if not os.path.exists(split_path):
            print(f"Verzeichnis nicht gefunden: {split_path}")
            continue

        for label_name, label_value in label_mapping.items():
            label_path = split_path + '\\' + label_name  # Auch hier explizit den backslash verwenden
            print(f"Überprüfe Unterordner: {label_path}")

            # Überprüfen, ob der Unterordner existiert
            if not os.path.exists(label_path):
                print(f"Unterordner nicht gefunden: {label_path}")
                continue

            # Dateien im Unterordner durchgehen
            for file_name in os.listdir(label_path):
                if file_name.endswith((".png", ".jpg", ".jpeg")):
                    file_path = label_path + '\\' + file_name  # Explizit backslash verwenden
                    print(f"Gefunden: {file_path} -> Label: {label_value}")
                    data.append({"file_path": file_path, "label": label_value})

    df = pd.DataFrame(data)
    return df

# Pfad zum Basisordner angeben
base_dir = r"C:\Users\leoni\Documents\HSLU\5. Semester\AI Einzelarbeit\Daten"

# Daten labeln
df = label_images(base_dir)
print(f"Anzahl der gefundenen Bilder: {len(df)}")
if len(df) > 0:
    print(df.head())
else:
    print("Keine Bilder gefunden!")


Verwendeter Basisordner: C:\Users\leoni\Documents\HSLU\5. Semester\AI Einzelarbeit\Daten
Überprüfe Ordner: C:\Users\leoni\Documents\HSLU\5. Semester\AI Einzelarbeit\Daten\Training
Verzeichnis nicht gefunden: C:\Users\leoni\Documents\HSLU\5. Semester\AI Einzelarbeit\Daten\Training
Überprüfe Ordner: C:\Users\leoni\Documents\HSLU\5. Semester\AI Einzelarbeit\Daten\Testing
Verzeichnis nicht gefunden: C:\Users\leoni\Documents\HSLU\5. Semester\AI Einzelarbeit\Daten\Testing
Anzahl der gefundenen Bilder: 0
Keine Bilder gefunden!
